In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymsm.plotting import stackplot
from pymsm.simulation import MultiStateSimulator, default_update_covariates_function

%load_ext autoreload
%autoreload 2

# Rossi - single state example

In [2]:
# Load rossi dataset
from lifelines.datasets import load_rossi

rossi = load_rossi()
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(rossi, duration_col="week", event_col="arrest")
baseline_hazard = cph.baseline_hazard_["baseline hazard"]
coefs = cph.params_

from pymsm.datasets import load_rossi_competing_risk_data

rossi_competing_risk_data, covariate_names = load_rossi_competing_risk_data()

### Define the full model ###
competing_risks_models_list = [
    {
        "origin_state": 1,
        "target_states": [2],
        "model_defs": [{"coefs": coefs, "baseline_hazard": baseline_hazard}],
    }
]

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[2],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_names,
)

# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=rossi_competing_risk_data.loc[0, covariate_names],
    origin_state=1,
    current_time=0,
    n_random_samples=1000,
    max_transitions=10,
    print_paths=False,
)


100%|██████████| 1000/1000 [00:01<00:00, 798.98it/s]


# 2 state example - AIDS-SI

In [3]:
from pymsm.datasets import load_aidssi

data = load_aidssi()
data.head()


# TODO 


,patnr,time,status,cause,ccr5
1,1,9.106,1,AIDS,WW
2,2,11.039,0,event-free,WM
3,3,2.234,1,AIDS,WW
4,4,9.878,2,SI,WM
5,5,3.819,1,AIDS,WW


# Many-state example - EBMT

In [4]:
# Fit model from data and extract model parts for simulation

from pymsm.datasets import prep_ebmt_long
competing_risk_dataset, covariate_cols, states_labels = prep_ebmt_long()

from pymsm.multi_state_competing_risks_model import MultiStateModel
multi_state_model = MultiStateModel(
    dataset=competing_risk_dataset,
    terminal_states = [5, 6],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_cols,
    competing_risk_data_format=True,
)

multi_state_model.fit()


Fitting Model at State: 1
>>> Fitting Transition to State: 2, n events: 785
>>> Fitting Transition to State: 3, n events: 907
>>> Fitting Transition to State: 5, n events: 95
>>> Fitting Transition to State: 6, n events: 160
Fitting Model at State: 2
>>> Fitting Transition to State: 5, n events: 112
>>> Fitting Transition to State: 6, n events: 39
>>> Fitting Transition to State: 4, n events: 227
Fitting Model at State: 3
>>> Fitting Transition to State: 4, n events: 433
>>> Fitting Transition to State: 6, n events: 197
>>> Fitting Transition to State: 5, n events: 56
Fitting Model at State: 4
>>> Fitting Transition to State: 5, n events: 107
>>> Fitting Transition to State: 6, n events: 137


In [8]:
from pymsm.simulation import extract_competing_risks_models_list_from_msm

competing_risks_models_list = extract_competing_risks_models_list_from_msm(
    multi_state_model, verbose=True
)

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[5, 6],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_cols,
)


1 2 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af155ddf0>
1 3 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af2dfe220>
1 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af2dfe040>
1 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af14e40a0>
2 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af149c2e0>
2 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af155d670>
2 4 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af157af70>
3 4 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af15125e0>
3 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af14a5e80>
3 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af14a5fd0>
4 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af095c8b0>
4 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7f3af095cf40>


In [10]:
# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=competing_risk_dataset.loc[0, covariate_cols],
    origin_state=1,
    current_time=0,
    n_random_samples=10,
    max_transitions=10,
    print_paths=True,
)

100%|██████████| 10/10 [00:00<00:00, 3530.86it/s]

[1, 3, 4, 6]
[20.000094023024143, 99.99994182354871, 4272.999964153427]
[1, 3, 5]
[23.00005203077009, 405.0000072106868]
[1, 3, 6]
[40.00004858717592, 39.000045085822954]
[1, 2, 5]
[24.00000891244321, 95.00002772374113]
[1, 2, 4, 6]
[19.000075513725566, 9.999926917871075, 2307.999997568403]
[1, 6]
[19.000023818278105]
[1, 2, 4, 6]
[21.0000862042651, 0.9999680601983734, 2196.9999457355366]
[1, 3, 5]
[21.000079729536576, 3015.9999202704635]
[1, 3, 4, 6]
[15.000069514455404, 53.999943887067445, 1508.999986598477]
[1, 2, 4, 6]
[19.000088420646335, 1.9999365085765781, 88.00000616900925]
